In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
from pprint import pprint
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/libraries/voxynth')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
%%yaml default_cfg

experiment:
  val_first: True 
  torch_compile: True 
  torch_mixed_precision: False 

dataloader:
  batch_size: '?'
  num_workers: '?' 
  pin_memory: True 

optim: # Unclear if we should tune this or not.
  _class: torch.optim.Adam
  lr: '?'

train:
  epochs: '?'
  eval_freq: '?'
  base_pretrained_dir: '?'
  use_pretrained_norm_augs: True 
  base_checkpoint: 'max-val-dice_score'
  base_pt_select_metric: 'val-dice_score'

loss_func: 
  _class: '?'

In [ ]:
%%yaml log_cfg

log:
  checkpoint_freq: 10
  root: '?'
  metrics:
    abs_area_estimation_error:
      _fn: ese.losses.area_estimation_error
      from_logits: True
      abs_diff: True

In [ ]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: 
          - abs_area_estimation_error
        phase: val

In [ ]:
%%yaml calibrator_defaults_cfg 

TS:
  _class: ese.models.calibrators.Temperature_Scaling

LTS:
  _class: ese.models.calibrators.LocalTS
  img_channels: 1
  num_classes: 1
  dims: '?'
  filters: [8, 8, 8]

3D_LTS_Huge:
  _class: ese.models.calibrators.LocalTS
  img_channels: 1
  num_classes: 1
  use_image: True
  abs_output: '?'
  dims: 3
  convs_per_block: 2
  filters: [64, 64, 64, 64, 64]

SCTS:
  _class: ese.models.resnet.SCTS
  dims: '?' 
  img_channels: 1
  num_classes: 1
  use_image: True 
  use_norm: True
  dims: 3
  # filters: [32, 32, 32]
  filters: [16, 16, 16]
  temp_range: (0.0, 3.0)
  blocks_per_layer: 2

E2T:
  _class: ese.models.peft.E2T
  in_features: 64
  num_classes: 1
  dims: 2
  features: [16, 16]
  temp_range: (0.0, 3.0)
  pool_fn: '?'


In [ ]:
%%yaml aug_cfg

augmentations:
    spatial:
        max_translation: 5.0
        max_rotation: 5.0
        max_scaling: 1.1
        warp_integrations: 5
        warp_smoothing_range: [10, 20]
        warp_magnitude_range: [1, 3]
        affine_probability: 0.5
        warp_probability: 0.5

In [ ]:
%%yaml experiment_cfg 

name: "OCTA_PredSweepSmall"

train: 
    base_pretrained_dir: "/storage/vbutoi/scratch/ESE/training/August_2024/08_07_24_OCTA_FULLRES_CrossEntropy/20240807_144627-M576-c3a81247e7e30c6bf47c72748764a898"

data:
    target: 'seg'

model:
    class_name: E2T 
    pool_fn: 'max'
    # pool_fn: 'mean'
  
optim:
    lr: 1.0e-2

dataloader:
    batch_size: 1
    num_workers: 1

loss_func:
    _class: ese.losses.AreaEstimationError 
    square_diff: True
    from_logits: True

In [ ]:
from ese.analysis.analysis_utils.submit_utils import get_ese_calibration_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([log_cfg, callbacks_cfg])

# For exp management, need a add_date variable because we reuse for func calls below.
add_date = True 
# Build the calibration configs from the options + base
updated_base_cfg, cal_cfgs = get_ese_calibration_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg,
    calibration_model_cfgs=calibrator_defaults_cfg,
    add_date=add_date
)

In [ ]:
len(cal_cfgs)

## Running Jobs

In [10]:
from ese.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

In [ ]:
####### Run individual jobs
run_ese_exp(
    config=cal_cfgs[0], 
    experiment_class=PostHocExperiment,
    run_name='debug',
    show_examples=True,
    track_wandb=False,
    gpu='4'
)

In [ ]:
# ### Run Batch Jobs
# submit_ese_exps(
#     group="calibration",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=cal_cfgs,
#     experiment_class=PostHocExperiment,
#     add_date=add_date,
#     track_wandb=True,
#     available_gpus=['0', '1', '2']
# )